# Cheryl Youtube History Analysis   

In [18]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Load the HTML file (adjust the file name/path as needed)
with open("CB_data/watch-history.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Inspect your HTML file to identify the right tags/classes.
# Here, we assume each history entry is within a div with class 'content-cell'
entries = soup.find_all("div", class_="content-cell")

data = []
for entry in entries:
    # Extract the title (YouTube video link text)
    title_tag = entry.find("a")
    title = title_tag.get_text(strip=True) if title_tag else None

    # Extract the URL (optional)
    url = title_tag["href"] if title_tag and "href" in title_tag.attrs else None

    # Extract the date/time
    time_str = entry.text.strip().split("\n")[-1]  # Extract last line, usually the date
    time_str = time_str.strip() if time_str else None

    # Try to parse the date (modify format if needed)
    date_time = None
    if time_str:
        try:
            date_time = datetime.strptime(time_str, "%B %d, %Y, %I:%M:%S %p UTC")
        except ValueError:
            date_time = time_str  # Keep original string if parsing fails

    # Only append if there's a valid title (to avoid extra records)
    if title:
        data.append({"title": title, "url": url, "date_time": date_time})

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,title,url,date_time
0,HALF HORSE HALF MAN | OFFICIAL VIDEO,https://www.youtube.com/watch?v=6v_R180kIGs,Watched HALF HORSE HALF MAN | OFFICIAL VIDEOOC...
1,here,https://myaccount.google.com/activitycontrols,Products: YouTubeWhy is this here? This activi...
2,Bluehost's ✨NEW✨ AI Website Builder,https://www.youtube.com/watch?v=u3UEiKiBlgE,Watched Bluehost's ✨NEW✨ AI Website BuilderFeb...
3,here,https://myaccount.google.com/activitycontrols,Products: YouTubeDetails: From Google AdsWhy i...
4,"[Solo-Leveling]SymphonicSuite-Lv.1 → Lv.2"" Mus...",https://www.youtube.com/watch?v=lmajFEi1Hdk,Watched [Solo-Leveling]SymphonicSuite-Lv.1 → L...


In [19]:
# fix the date and time parsing
import re
from datetime import datetime

def parse_date_time(text):
    # Use regex to match a valid date format starting with a month abbreviation.
    # This regex will match strings like "Feb 12, 2025, 11:53:21 PM EST"
    pattern = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4}, \d{1,2}:\d{2}:\d{2} [AP]M [A-Z]+'
    match = re.search(pattern, text)
    if match:
        date_str = match.group(0)
        print("Extracted date string:", date_str)
        # Define the expected format.
        # Note: The %Z directive (for timezone) may not correctly parse 'EST' in some environments.
        # You can remove the timezone if necessary.
        try:
            dt = datetime.strptime(date_str, "%b %d, %Y, %I:%M:%S %p %Z")
        except ValueError:
            # If parsing with timezone fails, try removing the timezone.
            dt = datetime.strptime(date_str.rsplit(" ", 1)[0], "%b %d, %Y, %I:%M:%S %p")
        return dt

df["date_time"] = df["date_time"].apply(parse_date_time)
df.head()
    


,title,url,date_time
0,HALF HORSE HALF MAN | OFFICIAL VIDEO,https://www.youtube.com/watch?v=6v_R180kIGs,None
1,here,https://myaccount.google.com/activitycontrols,None
2,Bluehost's ✨NEW✨ AI Website Builder,https://www.youtube.com/watch?v=u3UEiKiBlgE,None
3,here,https://myaccount.google.com/activitycontrols,None
4,"[Solo-Leveling]SymphonicSuite-Lv.1 → Lv.2"" Mus...",https://www.youtube.com/watch?v=lmajFEi1Hdk,None
